In [2]:
# AMATH 581 HW 1
# Jonathan McCormack 

import numpy as np
from scipy.integrate import solve_ivp

#-----------------------
# Exercise 1: Solve the ODE using Forward Euler and Heun's Methods
#-----------------------

# Define ODE
def f(y, t):
    return -3*y*np.sin(t)
    
# Define True Solution to ODE
def y_sol(t):
    return (np.pi/np.sqrt(2))*np.exp(3*(np.cos(t) -1))

# Define initial conditions and dt (step size)
y0 = np.pi/np.sqrt(2)
dt = np.power(2.0, np.arange(-2, -9, -1))

# Initialize error tracking
A2 = []
A5 = []
log_dt = []
log_error_FE = []
log_error_Heun = []

# Solve for each dt
for dt in dt:
    t = np.arange(0, 5+dt, dt)
    steps = len(t)

    # Initialize numerical solution
    y_num_FE = np.zeros(steps)
    y_num_FE[0] = y0
    y_num_Heun = np.zeros(steps)
    y_num_Heun[0] = y0

    # Calculate numerical solution using both methods 
    for i in range(steps - 1):
        y_num_FE[i+1] = y_num_FE[i] + dt*f(y_num_FE[i], t[i])
        y_num_Heun[i+1] = y_num_Heun[i] + (1/2)*dt*(f(y_num_Heun[i], t[i]) + f((y_num_Heun[i] + dt*f(y_num_Heun[i], t[i])),t[i+1]))
    
    # Calculate true solution for the current step size
    y_sol_t = y_sol(t)

    # Calculate mean error for each method
    error_FE = np.mean(np.abs(y_sol_t - y_num_FE))
    error_Heun = np.mean(np.abs(y_sol_t - y_num_Heun))
    
    # Store error values
    A2.append(error_FE)
    A5.append(error_Heun)

    # Store log values
    log_dt.append(np.log(dt))
    log_error_FE.append(np.log(error_FE))
    log_error_Heun.append(np.log(error_Heun))

    # Store numerical solution where dt = 2^-8
    if dt == 2**-8:
        A1 = y_num_FE
        A4 = y_num_Heun

# Reshape A1 and A4
A1 = A1.reshape(-1,1)
A4 = A4.reshape(-1,1)

# Derive A3 and A6 from plot
FE_coefficients = np.polyfit(log_dt, log_error_FE, 1)
A3 = FE_coefficients[0]
Heun_coefficients = np.polyfit(log_dt, log_error_Heun, 1)
A6 = Heun_coefficients[0]

del y0, dt, t, steps, y_num_FE, y_num_Heun, i, y_sol_t, error_FE, error_Heun, log_dt, log_error_FE, log_error_Heun, FE_coefficients, Heun_coefficients


#-----------------------
# Exercise 2: Solve the 2nd Order ODE (van der Pol oscillator) using solve_ivp from scipy.integrate
#-----------------------

# Define 2nd Order differential equation
def f(t, y1, y2, e):
    return -y1 - e*(y1**2-1)*y2
    
# Define System of 1st Order ODE
def system(t, Y):
    y1, y2 = Y # unpack dependent variables 
    dydt = [
        y2,           # dy1/dt = y2
        f(t, y1, y2, e)  # dy2/dt = f(t, y1, y2, e)
    ]
    return dydt

# Exercise 2a. Use solve_ivp for various epsilon values using RK45
# Define epsilon, step size, t_span, and initial conditions 
e = np.array([0.1, 1, 20])
t_eval = np.arange(0, 32.5, 0.5) # 0.5 step size
t_span = (t_eval[0], t_eval[-1]) # Solve from t=0 to t=32
initial_conditions = (np.sqrt(3), 1) # y(0)=sqrt(3), dy/dt(0)=1

# Initialize numerical solution
A7 = np.zeros(len(t_eval))
A7 = A7.reshape(-1,1)

# Solve for each epsilon
for e in e:
    sol = solve_ivp(system, t_span, initial_conditions, t_eval=t_eval)
    A7 = np.hstack((A7, ((sol.y[0,:]).reshape(-1,1))))
    
# Clean A7
A7 = np.delete(A7, 0, axis=1)

del e, t_eval, t_span, initial_conditions, sol
